# Code for Gillespie simulations

## Steps

- Input
  - Read the input matrix
  - create reactions
  - the update matrices
- Generate the steady state distribution
  - Every 300 time steps, give birth to a cell
  - simulate the cells in parallel


## Input


In [176]:
import pandas as pd
import numpy as np
import networkx as nx
import re

In [177]:
# Read the matrix
def read_input_matrix(path_to_matrix):
    """
    Reads the input matrix from the specified file path and counts number of genes.
    
    Parameters:
    path_to_matrix (str): The file path to the input matrix.
    
    Returns:
    np.ndarray: The input matrix as a NumPy array.
    """
    matrix = np.loadtxt(path_to_matrix, dtype='i', delimiter=',')
    if matrix.ndim == 0:
        matrix = np.array([[matrix]])

    # print(type(matrix))
    # print(matrix.shape)
    return matrix.shape[0], matrix

In [178]:
def generate_reaction_network_from_matrix(interaction_matrix):
    """
    Generate a reaction DataFrame directly from a signed interaction matrix.
    Assumes gene-specific parameters and interaction-specific regulation.

    Args:
        interaction_matrix (np.ndarray): shape (n_genes, n_genes)

    Returns:
        pd.DataFrame: reactions
        List[str]: gene names
    """
    n_genes = interaction_matrix.shape[0]
    gene_list = [f"gene_{i+1}" for i in range(n_genes)]

    prop = {
        "regulatory": "(({sign}*{p_add})*({activator}_protein**{n})/({k}**{n} + {activator}_protein**{n}))*{target}_I",
        "activation": "{p_on}*{target}_I",
        "inactivation": "{p_off}*{target}_A",
        "mRNA_prod": "{p_prod_mRNA}*{target}_A",
        "mRNA_deg": "{p_deg_mRNA}*{target}_mRNA",
        "protein_prod": "{p_prod_protein}*{target}_mRNA",
        "protein_deg": "{p_deg_protein}*{target}_protein"
    }

    reactions = []

    for j, target_gene in enumerate(gene_list):
        param = lambda p: f"{{{p}_{target_gene}}}"

        # Activation (gene_I → gene_A)
        expr = prop["activation"]
        expr = expr.replace("{p_on}", param("p_on")).replace("{target}", target_gene)
        reactions.append({
            "species1": f"{target_gene}_A", "change1": 1,
            "species2": f"{target_gene}_I", "change2": -1,
            "propensity": expr, "time": "-"
        })

        # Regulation by other genes (column j)
        regulators = np.where(interaction_matrix[:, j] != 0)[0]
        for i in regulators:
            source_gene = gene_list[i]
            sign = int(np.sign(interaction_matrix[i, j]))
            edge_tag = f"{source_gene}_to_{target_gene}"

            expr = prop["regulatory"]
            expr = expr.replace("{sign}", str(sign))
            expr = expr.replace("{p_add}", f"{{p_add_{edge_tag}}}")
            expr = expr.replace("{n}", f"{{n_{edge_tag}}}")
            expr = expr.replace("{k}", f"{{k_{edge_tag}}}")
            expr = expr.replace("{activator}", source_gene)
            expr = expr.replace("{target}", target_gene)

            reactions.append({
                "species1": f"{target_gene}_A", "change1": 1,
                "species2": f"{target_gene}_I", "change2": -1,
                "propensity": expr, "time": "-"
            })

        # Inactivation (gene_A → gene_I)
        expr = prop["inactivation"]
        expr = expr.replace("{p_off}", param("p_off")).replace("{target}", target_gene)
        reactions.append({
            "species1": f"{target_gene}_I", "change1": 1,
            "species2": f"{target_gene}_A", "change2": -1,
            "propensity": expr, "time": "-"
        })

        # Transcription & translation (uses gene-specific params)
        for label, suffix, change in [
            ("mRNA_prod", "mRNA", 1),
            ("mRNA_deg", "mRNA", -1),
            ("protein_prod", "protein", 1),
            ("protein_deg", "protein", -1)
        ]:
            expr = prop[label].replace("{target}", target_gene)
            for p in ["d", "p_prod_mRNA", "p_deg_mRNA", "p_prod_protein", "p_deg_protein"]:
                expr = expr.replace(f"{{{p}}}", param(p))
            reactions.append({
                "species1": f"{target_gene}_{suffix}", "change1": change,
                "species2": "-", "change2": "-",
                "propensity": expr, "time": "-"
            })

    # Consolidate reactions with same species1/species2/change values
    df = pd.DataFrame(reactions)
    df['propensity'] = df['propensity'].astype(str)
    reactions_df = (
        df.groupby(['species1', 'change1', 'species2', 'change2', 'time'])['propensity']
          .agg(lambda x: ' + '.join(x))
          .reset_index()
    )
    return reactions_df, gene_list

def generate_initial_state_from_genes(gene_list):
    """
    Generate an initial state where all genes are inactive and have zero mRNA/protein.

    Returns:
        pd.DataFrame with columns ['species', 'count']
    """
    states = []
    for gene in gene_list:
        states.extend([
            {"species": f"{gene}_A", "count": 0},
            {"species": f"{gene}_I", "count": 1},
            {"species": f"{gene}_mRNA", "count": 0},
            {"species": f"{gene}_protein", "count": 0}
        ])
    return pd.DataFrame(states)


In [179]:
def assign_parameters_to_genes(csv_path, n_genes, rows=None):
    """
    Assigns parameters from CSV to genes and returns a param_dict for expression substitution.
    
    Args:
        csv_path (str): Path to parameter CSV file
        rows (list of int, optional): Specific row indices to select. If None, selects randomly.
        n_random (int): Number of random rows to select if rows is None.
    
    Returns:
        tuple:
            param_dict (dict): {"{param_geneX}": value}
            param_matrix (pd.DataFrame): gene-wise parameter values
            row_mapping (dict): {"gene_X": row_index}
    """
    df = pd.read_csv(csv_path, index_col=0)

    # Select rows to assign to genes
    if rows is None:
        rows = np.random.choice(df.index, size=n_genes, replace=True)

    param_dict = {}
    param_matrix = {}
    row_mapping = {}

    for i, row in enumerate(rows):
        gene = f"gene_{i+1}"
        values = df.loc[row].copy()
        row_mapping[gene] = row

        # Derived params
        values["p_deg_mRNA"] = np.log(2) / values["mrna_half_life"]
        values["p_deg_protein"] = np.log(2) / values["protein_half_life"]
        values["p_prod_mRNA"] = values["burst_size"] * values["p_off"]

        # Remove unused columns
        values.drop(["mrna_half_life", "protein_half_life", "burst_size"], inplace=True, errors="ignore")

        # Add to param_matrix
        param_matrix[gene] = values

        # Flatten into param_dict with curly-brace keys
        for param, val in values.items():
            param_dict[f"{{{param}_{gene}}}"] = val

    param_matrix_df = pd.DataFrame(param_matrix).T
    return param_dict, param_matrix_df


In [180]:
# An useful utility function that is not used for simulation
# def calculate_unregulated_protein_levels(p_on, p_off, p_prod_mRNA, p_deg_mRNA, p_prod_protein, p_deg_protein, global_params=None):
#     """
#     Calculate protein levels without any regulation (for comparison/initialization).
    
#     Args:
#         param_matrix (pd.DataFrame): Gene parameters
#         global_params (dict, optional): Global constants
        
#     Returns:
#         np.ndarray: Unregulated steady-state protein levels
#     """    
#     # Calculate unregulated levels
#     burst_prob = p_on / (p_on + p_off)
#     mRNA = p_prod_mRNA * burst_prob / (p_deg_mRNA)
#     protein_levels = mRNA * p_prod_protein / p_deg_protein
    # return protein_levels

def generate_k_from_steady_state_calc(param_dict, interaction_matrix, gene_list,
                                        global_params=None, target_hill=0.5, scale_k=None, verbose=False):
    """
    Calculate steady-state protein levels using regulated k_on_eff for each gene,
    and update EC50 (k_*) values only for actual regulatory edges.

    Args:
        param_dict (dict): Parameter dictionary with gene-specific and edge-specific entries.
        interaction_matrix (np.ndarray): shape (n_genes, n_genes), effect of gene i on gene j.
        gene_list (list): List of gene names in order.
        global_params (dict): Optional global constants (unused).
        target_hill (float): Hill output to match when computing EC50.
        scale_k (np.ndarray or None): shape (n_genes, n_genes). scale_k[i, j] applies to k_{i→j}.
                                      Defaults to 1.0 for all entries.
        verbose (bool): Print debug info.

    Returns:
        tuple: (np.ndarray of steady-state protein levels, updated param_dict with k_* entries)
    """
    n_genes = len(gene_list)

    if scale_k is None:
        scale_k = np.ones((n_genes, n_genes))
    else:
        scale_k = np.asarray(scale_k)
        assert scale_k.shape == (n_genes, n_genes), "scale_k must be of shape (n_genes, n_genes)"

    protein_levels = np.zeros(n_genes)
    p_on_eff = np.zeros(n_genes)

    for i in range(n_genes):
        gene = gene_list[i]
        p_on = param_dict[f'{{p_on_{gene}}}']
        p_off = param_dict[f'{{p_off_{gene}}}']
        p_prod_mRNA = param_dict[f'{{p_prod_mRNA_{gene}}}']
        p_deg_mRNA = param_dict[f'{{p_deg_mRNA_{gene}}}']
        p_prod_protein = param_dict[f'{{p_prod_protein_{gene}}}']
        p_deg_protein = param_dict[f'{{p_deg_protein_{gene}}}']


        # Sum regulatory contributions
        regulatory_effect = 0.0
        regulators = np.where(interaction_matrix[:, i] != 0)[0]

        for reg in regulators:
            source = gene_list[reg]
            edge = f"{source}_to_{gene}"
            print(param_dict)
            p_add = param_dict.get(f"{{p_add_{edge}}}", 0)
            sign = interaction_matrix[reg, i]
            regulatory_effect += target_hill * p_add * sign
            print(source, edge, p_add, sign, regulatory_effect)

        p_on_eff[i] = p_on + regulatory_effect

        # # Compute protein level using k_on_eff
        # burst_prob = p_on_eff[i] / (p_on_eff[i] + p_off)
        # mRNA = p_prod_mRNA * burst_prob / p_deg_mRNA
        # protein = mRNA * p_prod_protein / p_deg_protein
        #Temp add 
        print(i, p_on, p_on_eff[i])
        burst_prob = p_on_eff[i] / (p_on_eff[i] + p_off)
        splicing_rate = np.log(2)/(7/60)
        unspliced_mRNA = p_prod_mRNA * burst_prob / (p_deg_mRNA + splicing_rate)
        spliced_mRNA = splicing_rate*unspliced_mRNA/(p_deg_mRNA)
        protein = spliced_mRNA * p_prod_protein / p_deg_protein
        protein_levels[i] = max(protein, 0.1)

        if verbose:
            print(f"Gene {gene}: k_on = {k_on:.3f} → k_on_eff = {k_on_eff[i]:.3f} "
                  f"(reg_effect: {regulatory_effect:.3f}) → Protein level: {protein_levels[i]:.3f}")

        # Assign EC50 values (k_*) only for actual edges, scaled with scale_k[i, j]
    for i in range(n_genes):
        source_gene = gene_list[i]
        targets = np.where(interaction_matrix[i, :] != 0)[0]
        for j in targets:
            target_gene = gene_list[j]
            key = f"{{k_{source_gene}_to_{target_gene}}}"
            param_dict[key] = protein_levels[i] * scale_k[i, j]

    return protein_levels, param_dict



In [181]:
def add_interaction_terms(param_dict, interaction_matrix, gene_list, n_matrix=None, p_add_matrix=None):
    """
    Add n and p_add terms to param_dict based on interaction_matrix.
    Also calculates EC50 (k) values using steady-state protein levels.

    Args:
        param_dict (dict): Initial dictionary of gene-specific parameters.
        interaction_matrix (np.ndarray): Regulatory interactions (2D array).
        gene_list (list): List of gene names like ['gene_1', 'gene_2', ...].
        n_matrix (np.ndarray, optional): Matrix of Hill coefficients (defaults to 2).
        p_add_matrix (np.ndarray, optional): Matrix of r_add values (defaults to 10).

    Returns:
        dict: Updated param_dict including n, p_add, and k values.
    """
    interaction_matrix = np.array(interaction_matrix)
    n_genes = len(gene_list)
    param_dict_updated = param_dict.copy()
    if n_matrix is None:
        n_matrix = np.full((n_genes, n_genes), 2)
    if p_add_matrix is None:
        p_add_matrix = np.full((n_genes, n_genes), 10)

    for i in range(n_genes):
        for j in range(n_genes):
            if interaction_matrix[i, j] != 0:
                gene_i = gene_list[i]
                gene_j = gene_list[j]
                edge = f"{gene_i}_to_{gene_j}"

                param_dict_updated[f"{{n_{edge}}}"] = n_matrix[i, j]
                param_dict_updated[f"{{p_add_{edge}}}"] = p_add_matrix[i, j]
    # Generate EC50 values using the correct steady-state calculation
    protein_levels, k_dict = generate_k_from_steady_state_calc(param_dict_updated, interaction_matrix, gene_list)

    return protein_levels, k_dict


In [182]:
def setup_gillespie_params_from_reactions(init_states, reactions, param_dictionary):
    """
    Setup Gillespie update matrix and function using species and parameter templates.

    Args:
        init_states (pd.DataFrame): columns ['species', 'count']
        reactions (pd.DataFrame): columns ['species1', 'change1', 'species2', 'change2', 'propensity', 'time']
        param_dictionary (dict): Dictionary containing all parameters (e.g., p_on_*, k_*, p_add_*, ...)

    Returns:
        population_init (np.ndarray), update_matrix (np.ndarray), update_function (str), species_index (dict)

    Raises:
        ValueError: If any required placeholder in reaction propensities is not in param_dictionary.
    """
    init_states = init_states.dropna()
    reactions = reactions.dropna()

    species_index = {s: i for i, s in enumerate(init_states['species'])}
    population_init = init_states['count'].values.astype(np.int64)
    
    update_matrix = []
    propensity_formulas = []
    missing_keys_report = []

    for i, row in reactions.iterrows():
        delta = [0] * len(species_index)
        delta[species_index[row['species1']]] = int(row['change1'])
        if row['species2'] != '-':
            delta[species_index[row['species2']]] = int(row['change2'])
        update_matrix.append(delta)

        expr = row['propensity']

        # Replace species with indexed population
        for species, idx in species_index.items():
            expr = expr.replace(species, f"population[{idx}]")

        # Validate and inject all parameter placeholders
        placeholders = set(re.findall(r"{[^}]+}", expr))
        missing = placeholders - set(param_dictionary.keys())
        if missing:
            missing_keys_report.append((i, row['propensity'], list(missing)))
            continue  # move to next reaction without injecting

        for key, val in param_dictionary.items():
            expr = expr.replace(key, str(val))

        if row['time'] != '-':
            line = f"propensities[{i}] = ({expr}) if ({row['time']}) else 0"
        else:
            line = f"propensities[{i}] = {expr}"
        propensity_formulas.append(line)

    if missing_keys_report:
        error_message = "Missing parameters in propensity expressions:\n"
        for i, raw_expr, missing_keys in missing_keys_report:
            error_message += f"  [Reaction {i}] '{raw_expr}' is missing: {', '.join(missing_keys)}\n"
        raise ValueError(error_message)

    update_func = "@numba.njit(fastmath=True)\ndef update_propensities(propensities, population, t):\n\t" + "\n\t".join(propensity_formulas)

    return population_init, np.array(update_matrix, dtype=np.int64), update_func, species_index


## Running the code


In [183]:
path_to_matrix = "/home/mzo5929/Keerthana/grnInference/simulation_data/general_simulation_code_data/test_data/matrix101.txt"
n_genes, interaction_matrix = read_input_matrix(path_to_matrix)
interaction_matrix

array([[0, 1],
       [0, 0]], dtype=int32)

In [184]:
reactions_df, gene_list = generate_reaction_network_from_matrix(interaction_matrix=interaction_matrix)

In [185]:
pd.set_option('display.max_colwidth', None)  # Set column width to unlimited
display(reactions_df)

,species1,change1,species2,change2,time,propensity
0,gene_1_A,1,gene_1_I,-1,-,{p_on_gene_1}*gene_1_I
1,gene_1_I,1,gene_1_A,-1,-,{p_off_gene_1}*gene_1_A
2,gene_1_mRNA,-1,-,-,-,{p_deg_mRNA_gene_1}*gene_1_mRNA
3,gene_1_mRNA,1,-,-,-,{p_prod_mRNA_gene_1}*gene_1_A
4,gene_1_protein,-1,-,-,-,{p_deg_protein_gene_1}*gene_1_protein
5,gene_1_protein,1,-,-,-,{p_prod_protein_gene_1}*gene_1_mRNA
6,gene_2_A,1,gene_2_I,-1,-,{p_on_gene_2}*gene_2_I + ((1*{p_add_gene_1_to_gene_2})*(gene_1_protein**{n_gene_1_to_gene_2})/({k_gene_1_to_gene_2}**{n_gene_1_to_gene_2} + gene_1_protein**{n_gene_1_to_gene_2}))*gene_2_I
7,gene_2_I,1,gene_2_A,-1,-,{p_off_gene_2}*gene_2_A
8,gene_2_mRNA,-1,-,-,-,{p_deg_mRNA_gene_2}*gene_2_mRNA
9,gene_2_mRNA,1,-,-,-,{p_prod_mRNA_gene_2}*gene_2_A


In [186]:
init_states = generate_initial_state_from_genes(gene_list=gene_list)
init_states

,species,count
0,gene_1_A,0
1,gene_1_I,1
2,gene_1_mRNA,0
3,gene_1_protein,0
4,gene_2_A,0
5,gene_2_I,1
6,gene_2_mRNA,0
7,gene_2_protein,0


In [187]:
param_path = "/home/mzo5929/Keerthana/grnInference/simulation_data/general_simulation_code_data/test_data/parameter_sheet_gillespie.csv"
rows = [0, 0]
param_dict, param_df = assign_parameters_to_genes(param_path, gene_list, rows)
param_dict

{'{p_on_gene_1}': 0.27,
 '{p_off_gene_1}': 8.4,
 '{p_prod_protein_gene_1}': 0.059,
 '{p_deg_mRNA_gene_1}': 0.2772588722239781,
 '{p_deg_protein_gene_1}': 0.024755256448569473,
 '{p_prod_mRNA_gene_1}': 268.8,
 '{p_on_gene_2}': 0.27,
 '{p_off_gene_2}': 8.4,
 '{p_prod_protein_gene_2}': 0.059,
 '{p_deg_mRNA_gene_2}': 0.2772588722239781,
 '{p_deg_protein_gene_2}': 0.024755256448569473,
 '{p_prod_mRNA_gene_2}': 268.8}

In [188]:
param_df

,p_on,p_off,p_prod_protein,p_deg_mRNA,p_deg_protein,p_prod_mRNA
gene_1,0.27,8.4,0.059,0.277259,0.024755,268.8
gene_2,0.27,8.4,0.059,0.277259,0.024755,268.8


In [189]:
p_add_matrix = np.array([
    [0.0, 2.0],  # gene_1 effects
    [0, 0]  # gene_2 effects
])

steady_state, full_param_dict = add_interaction_terms(param_dict, interaction_matrix, gene_list, p_add_matrix=p_add_matrix)

0 0.27 0.27
{'{p_on_gene_1}': 0.27, '{p_off_gene_1}': 8.4, '{p_prod_protein_gene_1}': 0.059, '{p_deg_mRNA_gene_1}': 0.2772588722239781, '{p_deg_protein_gene_1}': 0.024755256448569473, '{p_prod_mRNA_gene_1}': 268.8, '{p_on_gene_2}': 0.27, '{p_off_gene_2}': 8.4, '{p_prod_protein_gene_2}': 0.059, '{p_deg_mRNA_gene_2}': 0.2772588722239781, '{p_deg_protein_gene_2}': 0.024755256448569473, '{p_prod_mRNA_gene_2}': 268.8, '{n_gene_1_to_gene_2}': 2, '{p_add_gene_1_to_gene_2}': 2.0}
gene_1 gene_1_to_gene_2 2.0 1 1.0
1 0.27 1.27


In [190]:
steady_state

array([ 68.74872801, 289.93273173])

In [191]:
full_param_dict

{'{p_on_gene_1}': 0.27,
 '{p_off_gene_1}': 8.4,
 '{p_prod_protein_gene_1}': 0.059,
 '{p_deg_mRNA_gene_1}': 0.2772588722239781,
 '{p_deg_protein_gene_1}': 0.024755256448569473,
 '{p_prod_mRNA_gene_1}': 268.8,
 '{p_on_gene_2}': 0.27,
 '{p_off_gene_2}': 8.4,
 '{p_prod_protein_gene_2}': 0.059,
 '{p_deg_mRNA_gene_2}': 0.2772588722239781,
 '{p_deg_protein_gene_2}': 0.024755256448569473,
 '{p_prod_mRNA_gene_2}': 268.8,
 '{n_gene_1_to_gene_2}': 2,
 '{p_add_gene_1_to_gene_2}': 2.0,
 '{k_gene_1_to_gene_2}': 68.74872801372648}

In [192]:
f = setup_gillespie_params_from_reactions(init_states, reactions_df, full_param_dict)